# Initialisation de l'espace de travail

In [1]:
import pandas as pd
import numpy as np
import os
import re

BASE_DIR = os.getcwd()
DIR_NAME = os.path.basename(BASE_DIR)
print(BASE_DIR, DIR_NAME)
while DIR_NAME != "RakutenTeam": 
    BASE_DIR = os.path.realpath(os.path.join(os.path.dirname(__name__), '..'))
    DIR_NAME = os.path.basename(BASE_DIR)
    os.chdir(BASE_DIR)
    print( BASE_DIR, DIR_NAME)


c:\Users\mangg\projects\RakutenTeam\notebook notebook
C:\Users\mangg\projects\RakutenTeam RakutenTeam


In [2]:
DATA_PATH = os.path.join(BASE_DIR, "data")
IMG_PATH = os.path.join(DATA_PATH, "images")
IMG_TRAIN_PATH =  os.path.join(IMG_PATH, "image_train_resized")
IMG_TEST_PATH =  os.path.join(IMG_PATH, "image_test_resized")
CLEAN_DATA_PATH = os.path.join(DATA_PATH, "clean")
RAW_DATA_PATH = os.path.join(DATA_PATH, "raw")
CLEAN_TRAIN_DATA = os.path.join(CLEAN_DATA_PATH, "df_train.csv")
CLEAN_TEST_DATA = os.path.join(CLEAN_DATA_PATH, "df_test.csv")

# Travail de classification des Images par Deep Learning

## Préparation des données

#### Import des librairies, fonctions et classes requises

In [3]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, Dense, MaxPooling2D, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD

c:\Users\mangg\anaconda3\envs\Rakuten\lib\site-packages\tensorflow\python\framework\dtypes.py:513: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [4]:
train_data = pd.read_csv(CLEAN_TRAIN_DATA)
test_data = pd.read_csv(CLEAN_TEST_DATA)
print(train_data.shape, test_data.shape)
train_data.head()

(67932, 9) (16984, 9)


,designation,description,productid,imageid,language,designation_translated,description_translated,prdtypecode,prdtypedesignation
0,Porte Bébé Violet Et Rouge Trois-En-Un Mère Mu...,Porte bébé Violet et rouge Trois-en-un mère mu...,3050424970,1187504001,fr,Porte Bébé Violet Et Rouge Trois-En-Un Mère Mu...,Porte bébé Violet et rouge Trois-en-un mère mu...,1320,Puériculture
1,Jesus - Cahiers Du Libre Avenir,Prêtre autrement.,131641431,885888766,fr,Jesus - Cahiers Du Libre Avenir,Prêtre autrement.,10,Livres occasion
2,Chambre Paillasson En Forme De Coeur Tapis Flu...,Chambre Paillasson en forme de coeur Tapis Tap...,4197486437,1313030973,fr,Chambre Paillasson En Forme De Coeur Tapis Flu...,Chambre Paillasson en forme de coeur Tapis Tap...,1560,Mobilier
3,2pcs En Alliage D'aluminium Portail Du Carter ...,2pcs en alliage d'aluminium Portail du carter ...,3929174950,1265009801,fr,2pcs En Alliage D'aluminium Portail Du Carter ...,2pcs en alliage d'aluminium Portail du carter ...,1280,Jouets enfants
4,Harnais Chien Arnais Noir Anti Traction Gilet ...,La description: Fait de matériau souple doux ...,4183293159,1313455838,fr,Harnais Chien Arnais Noir Anti Traction Gilet ...,La description: Fait de matériau souple doux ...,2220,Animalerie


#### Ajout du Path au Dataframe de base

In [5]:
from src.features.images.transformers.path_finder import PathFinder

In [6]:
train_path_finder = PathFinder(img_folder=IMG_TRAIN_PATH, img_suffix="_resized")
train_data["path"] = train_path_finder.fit_transform(train_data)

test_path_finder = PathFinder(img_folder=IMG_TEST_PATH, img_suffix="_resized")
test_data["path"] = test_path_finder.fit_transform(test_data)

#### Encoding des labels

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
targets_train = encoder.fit_transform(train_data["prdtypedesignation"])
targets_test = encoder.transform(test_data["prdtypedesignation"])

#### Création des DataFrames Label/Img Path

In [8]:
img_data_train = pd.DataFrame({"label": targets_train, "img_path": train_data["path"]})
img_data_test = pd.DataFrame({"label": targets_test, "img_path": test_data["path"]})

In [9]:
img_data_train.head()

,label,img_path
0,24,C:\Users\mangg\projects\RakutenTeam\data\image...
1,17,C:\Users\mangg\projects\RakutenTeam\data\image...
2,20,C:\Users\mangg\projects\RakutenTeam\data\image...
3,13,C:\Users\mangg\projects\RakutenTeam\data\image...
4,1,C:\Users\mangg\projects\RakutenTeam\data\image...


## Modélisation ResNet50

#### Import des librairies requises


In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50